In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import nibabel as nib
import pcntoolkit as ptk 
from pcntoolkit.dataio.fileio import load as ptkload
from pcntoolkit.dataio.fileio import save as ptksave
from pcntoolkit.dataio.fileio import save_nifti, load_nifti
from pcntoolkit.normative import predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix, calibration_descriptives

from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
import pingouin as pg

#%% SET GLOBALS
# globals
root_dir = '/project_cephfs/3022017.06/ENIGMA_ANX/'


###  CHANGE DEPENDING ON Z-STAT OR SCALED EFFECT  ###
proc_dir = os.path.join(root_dir,'Z_stat/')
#proc_dir = os.path.join(root_dir,'Scaled_effect/')

data_dir = os.path.join(proc_dir,'data/')
w_dir = os.path.join(proc_dir,'vox/')
mask_nii = ('/opt/fmriprep/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz')
ex_nii = os.path.join(data_dir, 'ENIGMA_FC_tr_1.nii.gz')



/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#Load data
Z_est_clinical = ptkload(os.path.join(w_dir,'Z_predcl.pkl'), mask=mask_nii)

#Load in the diagnosis information
metadata_cl_diagnosis = pd.read_csv('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/data/clinical_te.csv')

In [3]:
#COMPUTE AND SAVE COUNTS
mask_Diganosis = metadata_cl_diagnosis['Healthy_or_patient'].eq(1)
mask_Control = metadata_cl_diagnosis['Healthy_or_patient'].eq(0)
Z_est_clinical_only = Z_est_clinical[mask_Diganosis]
Z_est_control_clin = Z_est_clinical[mask_Control]


ptksave(Z_est_clinical_only, os.path.join(w_dir,'Z_predcl_CLINICAL.nii.gz'), example=ex_nii, mask=mask_nii)
ptksave(Z_est_control_clin, os.path.join(w_dir,'Z_predcl_CONTROL.nii.gz'), example=ex_nii, mask=mask_nii)

print('DONE')

DONE


In [4]:
#THRESHOLD THE NII FILES USING FSLMATHS - control

in_filename = (w_dir +'Z_predcl_CONTROL.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_predcl_CONTROL_pos2pt6.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CONTROL_neg2pt6.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename +' -thr 2.6 ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename +' -mul -1 -thr 2.6 ' +out_filename_neg)
print(command)
os.system(command)
!command


fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL.nii.gz -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_pos2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL.nii.gz -mul -1 -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_neg2pt6.nii.gz


In [5]:
#THRESHOLD THE NII FILES USING FSLMATHS - clinical

in_filename = (w_dir +'Z_predcl_CLINICAL.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_predcl_CLINICAL_pos2pt6.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CLINICAL_neg2pt6.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename +' -thr 2.6 ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename +' -mul -1 -thr 2.6 ' +out_filename_neg)
print(command)
os.system(command)
!command


fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL.nii.gz -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_pos2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL.nii.gz -mul -1 -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_neg2pt6.nii.gz


In [6]:
# BINARISE THE NII FILES USING FSLMATHS
in_filename_pos = (w_dir +'Z_predcl_CONTROL_pos2pt6.nii.gz')
in_filename_neg = (w_dir +'Z_predcl_CONTROL_neg2pt6.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_predcl_CONTROL_pos2pt6_bin.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CONTROL_neg2pt6_bin.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename_pos +' -bin  ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename_neg +' -bin  ' +out_filename_neg)
print(command)
os.system(command)
!command
print('DONE')

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_pos2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_pos2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_neg2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_neg2pt6_bin.nii.gz
DONE


In [9]:
# BINARISE THE NII FILES USING FSLMATHS
in_filename_pos = (w_dir +'Z_predcl_CLINICAL_pos2pt6.nii.gz')
in_filename_neg = (w_dir +'Z_predcl_CLINICAL_neg2pt6.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_predcl_CLINICAL_pos2pt6_bin.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CLINICAL_neg2pt6_bin.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename_pos +' -bin  ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename_neg +' -bin  ' +out_filename_neg)
print(command)
os.system(command)
!command
print('DONE')

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_pos2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_pos2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_neg2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_neg2pt6_bin.nii.gz
DONE


In [7]:
# SUM THE NUMBER OF VOXELS
in_filename_pos = (w_dir +'Z_predcl_CONTROL_pos2pt6_bin.nii.gz')
in_filename_neg = (w_dir +'Z_predcl_CONTROL_neg2pt6_bin.nii.gz')
thresh_bin_nii_pos = load_nifti(in_filename_pos, mask=mask_nii)
thresh_bin_nii_neg = load_nifti(in_filename_neg, mask=mask_nii)

out_filename_pos = (w_dir +'Z_predcl_CONTROL_count_pos.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CONTROL_count_neg.nii.gz')
#print(out_filename_pos, out_filename_neg)
   
#Sum along the 2nd dimension(4D = volumes = participants) of the Positive thresholded, binarised file
sum_site_pos = np.sum(thresh_bin_nii_pos,axis=1)
#Save as nii
save_nifti(sum_site_pos,out_filename_pos, examplenii=mask_nii, mask=mask_nii, dtype='float32')

#Sum along the 2nd dimension(4D = volumes = participants) of the Negative thresholded, binarised file
sum_site_neg = np.sum(thresh_bin_nii_neg,axis=1)
#Save as nii
save_nifti(sum_site_neg,out_filename_neg, examplenii=mask_nii, mask=mask_nii, dtype='float32')

print('DONE')

DONE


In [11]:
# SUM THE NUMBER OF VOXELS
in_filename_pos = (w_dir +'Z_predcl_CLINICAL_pos2pt6_bin.nii.gz')
in_filename_neg = (w_dir +'Z_predcl_CLINICAL_neg2pt6_bin.nii.gz')
thresh_bin_nii_pos = load_nifti(in_filename_pos, mask=mask_nii)
thresh_bin_nii_neg = load_nifti(in_filename_neg, mask=mask_nii)

out_filename_pos = (w_dir +'Z_predcl_CLINICAL_count_pos.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CLINICAL_count_neg.nii.gz')
#print(out_filename_pos, out_filename_neg)
   
#Sum along the 2nd dimension(4D = volumes = participants) of the Positive thresholded, binarised file
sum_site_pos = np.sum(thresh_bin_nii_pos,axis=1)
#Save as nii
save_nifti(sum_site_pos,out_filename_pos, examplenii=mask_nii, mask=mask_nii, dtype='float32')

#Sum along the 2nd dimension(4D = volumes = participants) of the Negative thresholded, binarised file
sum_site_neg = np.sum(thresh_bin_nii_neg,axis=1)
#Save as nii
save_nifti(sum_site_neg,out_filename_neg, examplenii=mask_nii, mask=mask_nii, dtype='float32')

print('DONE')

DONE


Re-save Clinical for just the affective patients

In [8]:
#Load in the diagnosis information
metadata_cl_diagnosis = pd.read_csv('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/data/clinical_te.csv')
Diagnoses_info = pd.read_csv('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/data/all_test_validation.csv', usecols = ['SubjectID_ENIGMA', 'Principal_diagnosis_current'] )
merged_clinical_test_data = pd.merge(metadata_cl_diagnosis, Diagnoses_info, on='SubjectID_ENIGMA', how='left')

merged_clinical_test_data['Principal_diagnosis_current'] = merged_clinical_test_data['Principal_diagnosis_current'].replace({np.nan: 'NA', 'nan': 'NA'})
Primary_Diagnoses = np.unique(merged_clinical_test_data['Principal_diagnosis_current'])

#split the Z_est clinical into the control and patients  ***and remove other and schizophrenia as diagnosis are not of interest***
mask_Diganosis = merged_clinical_test_data['Healthy_or_patient'].eq(1)
mask_exclude_diagnosis = ~merged_clinical_test_data['Principal_diagnosis_current'].isin(['others', 'schizophrenia'])
 #remove other and schizophrenia as not primary affective conditions 
combined_Diagnosis_mask = mask_Diganosis & mask_exclude_diagnosis

Diagnoses_info = merged_clinical_test_data[combined_Diagnosis_mask]
Affective_sample_deviations = Z_est_clinical[combined_Diagnosis_mask]
print(len(Affective_sample_deviations))

ptksave(Affective_sample_deviations, os.path.join(w_dir,'Z_predcl_CLINICAL_affective_only.nii.gz'), example=ex_nii, mask=mask_nii)


222


In [10]:
#THRESHOLD THE NII FILES USING FSLMATHS - clinical

in_filename = (w_dir +'Z_predcl_CLINICAL_affective_only.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_predcl_CLINICAL_affective_only_pos2pt6.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CLINICAL_affective_only_neg2pt6.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename +' -thr 2.6 ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename +' -mul -1 -thr 2.6 ' +out_filename_neg)
print(command)
os.system(command)
!command


fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only.nii.gz -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only_pos2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only.nii.gz -mul -1 -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only_neg2pt6.nii.gz


In [11]:
# BINARISE THE NII FILES USING FSLMATHS
in_filename_pos = (w_dir +'Z_predcl_CLINICAL_affective_only_pos2pt6.nii.gz')
in_filename_neg = (w_dir +'Z_predcl_CLINICAL_affective_only_neg2pt6.nii.gz')
#print(filename)
out_filename_pos = (w_dir +'Z_predcl_CLINICAL_affective_only_pos2pt6_bin.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CLINICAL_affective_only_neg2pt6_bin.nii.gz')
#print(out_filename_pos, out_filename_neg)

command = ('fslmaths ' +in_filename_pos +' -bin  ' +out_filename_pos)
print(command)
os.system(command)
!command

command = ('fslmaths ' +in_filename_neg +' -bin  ' +out_filename_neg)
print(command)
os.system(command)
!command
print('DONE')

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only_pos2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only_pos2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only_neg2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CLINICAL_affective_only_neg2pt6_bin.nii.gz
DONE


In [12]:
# SUM THE NUMBER OF VOXELS
in_filename_pos = (w_dir +'Z_predcl_CLINICAL_affective_only_pos2pt6_bin.nii.gz')
in_filename_neg = (w_dir +'Z_predcl_CLINICAL_affective_only_neg2pt6_bin.nii.gz')
thresh_bin_nii_pos = load_nifti(in_filename_pos, mask=mask_nii)
thresh_bin_nii_neg = load_nifti(in_filename_neg, mask=mask_nii)

out_filename_pos = (w_dir +'Z_predcl_CLINICAL_affective_only_count_pos.nii.gz')
out_filename_neg = (w_dir +'Z_predcl_CLINICAL_affective_only_count_neg.nii.gz')
#print(out_filename_pos, out_filename_neg)
   
#Sum along the 2nd dimension(4D = volumes = participants) of the Positive thresholded, binarised file
sum_site_pos = np.sum(thresh_bin_nii_pos,axis=1)
#Save as nii
save_nifti(sum_site_pos,out_filename_pos, examplenii=mask_nii, mask=mask_nii, dtype='float32')

#Sum along the 2nd dimension(4D = volumes = participants) of the Negative thresholded, binarised file
sum_site_neg = np.sum(thresh_bin_nii_neg,axis=1)
#Save as nii
save_nifti(sum_site_neg,out_filename_neg, examplenii=mask_nii, mask=mask_nii, dtype='float32')

print('DONE')

DONE


Re-save Controls as a single file

In [13]:
#Sum across the two pos count files together
command = ('fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_count_pos.nii.gz -add /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_count_pos.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/ALL_CONTROLS_pos_count.nii.gz')
print(command)
os.system(command)
!command
print('DONE pos')

#Sum across the two neg count files together
command = ('fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_count_neg.nii.gz -add /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_count_neg.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/ALL_CONTROLS_neg_count.nii.gz')
print(command)
os.system(command)
!command
print('DONE neg')


fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_count_pos.nii.gz -add /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_count_pos.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/ALL_CONTROLS_pos_count.nii.gz
DONE pos
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_ref_count_neg.nii.gz -add /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/Z_predcl_CONTROL_count_neg.nii.gz /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/ALL_CONTROLS_neg_count.nii.gz
DONE neg
